In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tiktok-github-luminati-io-data/README.md
/kaggle/input/tiktok-github-luminati-io-data/tiktok-profiles.csv


In [2]:
df = pd.read_csv('/kaggle/input/tiktok-github-luminati-io-data/tiktok-profiles.csv')

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
df.head(2)

,account_id,nickname,biography,awg_engagement_rate,comment_engagement_rate,like_engagement_rate,bio_link,predicted_lang,is_verified,followers,...,comment_setting,duet_setting,stitch_setting,is_ad_virtual,room_id,is_under_age_18,region,top_videos,signature,discovery_input
0,jzt███_ar███,jztang_art,Tattoo artist in LA\nIG @jztang_art_modoink\nA...,1.619883,0.011696,1.608187,NaN,en,False,19,...,NaN,NaN,NaN,False,NaN,NaN,US,"[""https://www.tiktok.com/@jztang_art/video/693...",Tattoo artist in LA\nIG @jztang_art_modoink\nA...,NaN
1,amw███qna███,🦋Emanuelle Maia🦋,Aquele em que ela ama futebol e Harry Potter 🥰...,0.071843,0.005806,0.066038,NaN,en,False,106,...,NaN,NaN,NaN,False,NaN,NaN,BR,"[""https://www.tiktok.com/@amwbaiqnajaj/video/6...",Aquele em que ela ama futebol e Harry Potter 🥰...,NaN


1. account_id doesn't give any useful information. Nevertheless most of it is hidden. So dropping that column will be better. 
2. nickname doesn't have any null values
3. biography doesn't give us much info ... random, just like bios are
4. awg_engagement_rate: The average engagement rate for the user's content
5. comment_engagement_rate: The engagement rate specifically related to comments on the user's content
6. like_engagement_rate: The engagement rate specifically related to likes on the user's content
these could be null
7. bio link: not informative
8. predicted_lang: lang of the user's content
9. is_verified: T/F
10. followers, following, likes videos_count: numerical
11. create time : drop
12. id: A unique identifier associated with the TikTok user
13. url: The URL or link to the TikTok user's profile -- not informative same for profile_pic_url
14. is_ad_virtual: Informative
15. region

We can drop a lot of columns 

In [5]:
df.columns

Index(['account_id', 'nickname', 'biography', 'awg_engagement_rate',
       'comment_engagement_rate', 'like_engagement_rate', 'bio_link',
       'predicted_lang', 'is_verified', 'followers', 'following', 'likes',
       'videos_count', 'create_time', 'id', 'url', 'profile_pic_url',
       'like_count', 'digg_count', 'is_private', 'profile_pic_url_hd',
       'secu_id', 'short_id', 'ftc', 'relation', 'open_favorite',
       'comment_setting', 'duet_setting', 'stitch_setting', 'is_ad_virtual',
       'room_id', 'is_under_age_18', 'region', 'top_videos', 'signature',
       'discovery_input'],
      dtype='object')

What is diggCount on TikTok?

Correlations between number of video plays ('playCount'), number of comments ('commentCount'), number of likes ('diggCount'), and number of shares ('shareCount'). Each variable is crucial for defining the amount of video engagement on TikTok.

In [6]:
# Drop unnecessary columns
data = df[['id', 'awg_engagement_rate', 'comment_engagement_rate', 'like_engagement_rate', 
                  'predicted_lang', 'is_verified', 'followers', 'following', 'likes', 
                  'videos_count', 'digg_count', 'is_private', 'is_ad_virtual', 'region', 'top_videos']].copy()

data.head()

,id,awg_engagement_rate,comment_engagement_rate,like_engagement_rate,predicted_lang,is_verified,followers,following,likes,videos_count,digg_count,is_private,is_ad_virtual,region,top_videos
0,6744469562807747589,1.619883,0.011696,1.608187,en,False,19,1,275,9,0,NaN,False,US,"[""https://www.tiktok.com/@jztang_art/video/693..."
1,6904794783643223046,0.071843,0.005806,0.066038,en,False,106,378,1286,13,0,NaN,False,BR,"[""https://www.tiktok.com/@amwbaiqnajaj/video/6..."
2,6566918905926762501,0.003516,0.000034,0.003482,en,False,5826,5,793000,1537,0,NaN,False,SA,"[""https://www.tiktok.com/@k.mssqwziiii/video/7..."
3,6637060722835046405,0.001500,0.000036,0.001464,en,False,44600,97,224700,2091,0,NaN,False,BD,"[""https://www.tiktok.com/@towwnimaaa_/video/72..."
4,6804509823943214085,6.154908,0.042945,6.111963,en,False,163,53,7552,12,0,NaN,False,RU,"[""https://www.tiktok.com/@_uiui_r_/video/70719..."


In [7]:
# check missing values 
data.isnull().sum()

id                           0
awg_engagement_rate          0
comment_engagement_rate      0
like_engagement_rate         0
predicted_lang               0
is_verified                  0
followers                    0
following                    0
likes                        0
videos_count                 0
digg_count                   0
is_private                 717
is_ad_virtual                0
region                       3
top_videos                   0
dtype: int64

In [8]:
data.is_private.value_counts()

is_private
False    283
Name: count, dtype: int64

In [9]:
data['is_private'].fillna(True, inplace = True)
data.isnull().sum()

id                         0
awg_engagement_rate        0
comment_engagement_rate    0
like_engagement_rate       0
predicted_lang             0
is_verified                0
followers                  0
following                  0
likes                      0
videos_count               0
digg_count                 0
is_private                 0
is_ad_virtual              0
region                     3
top_videos                 0
dtype: int64

In [10]:
data.region.value_counts()

region
US    159
ID     58
IQ     56
SA     54
VN     38
     ... 
IE      1
LK      1
SN      1
GH      1
PA      1
Name: count, Length: 104, dtype: int64

In [11]:
data['region'].fillna(data['region'].mode()[0], inplace = True)
data.isnull().sum()

id                         0
awg_engagement_rate        0
comment_engagement_rate    0
like_engagement_rate       0
predicted_lang             0
is_verified                0
followers                  0
following                  0
likes                      0
videos_count               0
digg_count                 0
is_private                 0
is_ad_virtual              0
region                     0
top_videos                 0
dtype: int64

In [12]:
data.describe()

,id,awg_engagement_rate,comment_engagement_rate,like_engagement_rate,followers,following,likes,videos_count,digg_count
count,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000
mean,6.598018e+18,0.600419,0.036881,0.563538,14260.810000,1180.816000,2.546741e+05,144.653000,1763.996000
std,1.331342e+18,3.042836,0.222862,2.969609,60002.045531,2001.699544,1.567335e+06,314.819898,8746.779728
min,7.026413e+06,0.000039,0.000000,0.000037,1.000000,0.000000,2.000000e+00,1.000000,0.000000
25%,6.749596e+18,0.023019,0.000823,0.021006,167.000000,84.750000,4.402500e+02,9.000000,0.000000
50%,6.871683e+18,0.085272,0.003159,0.080300,871.500000,338.000000,3.520000e+03,38.000000,0.000000
75%,6.986654e+18,0.287537,0.012839,0.264512,3240.000000,1243.500000,2.912500e+04,139.000000,0.000000
max,7.257614e+18,80.576923,4.000000,80.034615,812300.000000,10000.000000,2.840000e+07,3923.000000,101400.000000


In [13]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Select the columns to standardize
columns_to_standardize = ['awg_engagement_rate', 'comment_engagement_rate', 'like_engagement_rate']

# Standardization (Z-score)
scaler = StandardScaler()
data[columns_to_standardize] = scaler.fit_transform(data[columns_to_standardize])

# Check results
display(data[columns_to_standardize].describe())

,awg_engagement_rate,comment_engagement_rate,like_engagement_rate
count,1.000000e+03,1.000000e+03,1.000000e+03
mean,-1.598721e-17,3.552714e-17,-1.598721e-17
std,1.000500e+00,1.000500e+00,1.000500e+00
min,-1.974082e-01,-1.655691e-01,-1.898510e-01
25%,-1.898521e-01,-1.618728e-01,-1.827864e-01
50%,-1.693830e-01,-1.513885e-01,-1.628093e-01
75%,-1.028772e-01,-1.079314e-01,-1.007459e-01
max,2.629669e+01,1.779176e+01,2.677485e+01


In [14]:
data.dtypes

id                           int64
awg_engagement_rate        float64
comment_engagement_rate    float64
like_engagement_rate       float64
predicted_lang              object
is_verified                   bool
followers                    int64
following                    int64
likes                        int64
videos_count                 int64
digg_count                   int64
is_private                    bool
is_ad_virtual                 bool
region                      object
top_videos                  object
dtype: object

In [15]:
data.head()

,id,awg_engagement_rate,comment_engagement_rate,like_engagement_rate,predicted_lang,is_verified,followers,following,likes,videos_count,digg_count,is_private,is_ad_virtual,region,top_videos
0,6744469562807747589,0.335205,-0.113062,0.351956,en,False,19,1,275,9,0,True,False,US,"[""https://www.tiktok.com/@jztang_art/video/693..."
1,6904794783643223046,-0.173798,-0.139506,-0.167615,en,False,106,378,1286,13,0,True,False,BR,"[""https://www.tiktok.com/@amwbaiqnajaj/video/6..."
2,6566918905926762501,-0.196265,-0.165415,-0.188690,en,False,5826,5,793000,1537,0,True,False,SA,"[""https://www.tiktok.com/@k.mssqwziiii/video/7..."
3,6637060722835046405,-0.196928,-0.165408,-0.189370,en,False,44600,97,224700,2091,0,True,False,BD,"[""https://www.tiktok.com/@towwnimaaa_/video/72..."
4,6804509823943214085,1.826345,0.027224,1.869337,en,False,163,53,7552,12,0,True,False,RU,"[""https://www.tiktok.com/@_uiui_r_/video/70719..."


## The data is clean, let's proceed with SQL queries

In [16]:
import sqlite3
# Create an SQLite database in memory
conn = sqlite3.connect(":memory:")  #  creates an in-memory SQLite database
cursor = conn.cursor()  # Explicitly create a cursor 

# Load cleaned dataset into SQLite
data.to_sql("tiktok_profiles", conn, index=False, if_exists="replace")

1000

### Top 10 Influencers by Engagement Rate

In [17]:
query = """
select id, followers, videos_count, awg_engagement_rate
from tiktok_profiles 
order by awg_engagement_rate desc 
limit 10
"""
top_influencers = pd.read_sql(query, conn)
display(top_influencers)

,id,followers,videos_count,awg_engagement_rate
0,6761860606440375297,130,2,26.296690
1,6989677079177397254,1,2,6.378683
2,6791545190252495878,870,6,5.778236
3,7066131924145701894,1157,4,5.503385
4,7072780776455390214,12,3,4.122714
5,7016420040141456389,3,2,3.967445
6,6733545369197757441,178,17,3.930790
7,6949875187790660613,1,1,3.419436
8,6948148615328793605,3202,18,3.264658
9,6951881032375436293,19800,1,3.243409


Insight: Identifies influencers with highest engagement rates.


### Followers vs. Engagement Rate by Influencer Category

In [18]:
query = """
select 
case 
WHEN followers < 10000 THEN 'Micro'
WHEN followers BETWEEN 10000 AND 100000 THEN 'Mid-tier'
WHEN followers BETWEEN 100000 AND 1000000 THEN 'Macro'
ELSE 'Mega'
end as influencer_category, avg(awg_engagement_rate) as avg_engagement
from tiktok_profiles 
group by influencer_category
"""
engagement_by_category = pd.read_sql(query, conn)
display(engagement_by_category)

,influencer_category,avg_engagement
0,Macro,-0.175344
1,Micro,0.019485
2,Mid-tier,-0.093232


Do micro-influencers have better engagement than macro-influencers?

No

### Top 10 Regions with Highest Engagement

In [19]:
query = """
SELECT region, COUNT(id) AS num_influencers, AVG(awg_engagement_rate) AS avg_engagement
FROM tiktok_profiles
GROUP BY region
ORDER BY avg_engagement DESC
LIMIT 10;
"""
top_regions = pd.read_sql(query, conn)
display(top_regions)

,region,num_influencers,avg_engagement
0,NO,2,1.614797
1,RO,4,0.945466
2,CL,3,0.889015
3,NP,2,0.839284
4,XK,1,0.778035
5,DK,1,0.725699
6,ES,11,0.551023
7,ID,58,0.373620
8,CO,6,0.367992
9,FR,24,0.345772


### Engagement Rate of Verified vs. Non-Verified Influencers

In [20]:
query = """
SELECT is_verified, COUNT(id) AS num_influencers,
    AVG(awg_engagement_rate) AS avg_engagement,
    AVG(followers) AS avg_followers
FROM tiktok_profiles
GROUP BY is_verified;
"""
verified_stats = pd.read_sql(query, conn)
display(verified_stats)

,is_verified,num_influencers,avg_engagement,avg_followers
0,0,998,0.000374,13734.178357
1,1,2,-0.186414,277050.000000


### Relationship Between Video Count & Engagement

In [21]:
query = """
SELECT videos_count, AVG(awg_engagement_rate) AS avg_engagement
FROM tiktok_profiles
GROUP BY videos_count
ORDER BY videos_count DESC
LIMIT 20;
"""
video_engagement = pd.read_sql(query, conn)
display(video_engagement)


,videos_count,avg_engagement
0,3923,-0.196179
1,2635,-0.197408
2,2495,-0.193987
3,2233,-0.197230
4,2204,-0.197097
5,2122,-0.196758
6,2091,-0.196928
7,1956,-0.197255
8,1917,-0.194298
9,1908,-0.196478


### Does verification improve engagement?

In [22]:
query = """
SELECT is_verified, 
       AVG(awg_engagement_rate) AS avg_engagement, 
       COUNT(id) AS num_influencers
FROM tiktok_profiles
GROUP BY is_verified;
"""
ab_test_result = pd.read_sql(query, conn)
display(ab_test_result)


,is_verified,avg_engagement,num_influencers
0,0,0.000374,998
1,1,-0.186414,2


Non-verified influencers generally have a very low engagement rate (close to zero).

This negative value suggests that after standardization, the verified influencers' engagement rate is below the dataset's mean engagement rate.

### Marketplace Efficiency - Follower Growth Analysis

Measure the growth rate of followers to identify fast-rising influencers.
Helps TikTok optimize its influencer marketplace.

In [23]:
query = """
SELECT id, followers, videos_count, 
       ROUND((followers / NULLIF(videos_count, 0)), 2) AS followers_per_video
FROM tiktok_profiles
ORDER BY followers_per_video DESC
LIMIT 10;
"""
follower_growth = pd.read_sql(query, conn)
display(follower_growth)


,id,followers,videos_count,followers_per_video
0,6951881032375436293,19800,1,19800.0
1,6798230807473849350,15800,1,15800.0
2,6911061574435357698,24800,2,12400.0
3,81028050645106688,487800,67,7280.0
4,6948829470229300229,415000,63,6587.0
5,6714593971282641926,5804,1,5804.0
6,6808092341582267394,722300,153,4720.0
7,6762674634293019653,166100,36,4613.0
8,6964313100094866433,387900,90,4310.0
9,73008986968,63100,15,4206.0


### What Type of Content Gets the Most Engagement?

In [24]:
query = """
SELECT 
    SUBSTR(top_videos, 1, 50) AS sample_video,
    COUNT(id) AS num_influencers,
    ROUND(AVG(awg_engagement_rate), 4) AS avg_engagement
FROM tiktok_profiles
GROUP BY sample_video
ORDER BY avg_engagement DESC
LIMIT 5;
"""
engagement = pd.read_sql(query, conn)
display(engagement)

,sample_video,num_influencers,avg_engagement
0,"[""https://www.tiktok.com/@sindiicn/video/70812...",1,26.2967
1,"[""https://www.tiktok.com/@ltlt...ltlt/video/69...",1,6.3787
2,"[""https://www.tiktok.com/@parrica08/video/7258...",1,5.7782
3,"[""https://www.tiktok.com/@tclsfilms/video/7067...",1,5.5034
4,"[""https://www.tiktok.com/@p4ul4x/video/7073542...",1,4.1227


### Fastest Growing Influencers (Followers per Video)

In [25]:
query = """
SELECT id, followers, videos_count,
       ROUND((followers / NULLIF(videos_count, 0)), 2) AS followers_per_video
FROM tiktok_profiles
ORDER BY followers_per_video DESC
LIMIT 10;

"""

fast_influencer = pd.read_sql(query, conn)
display(fast_influencer)

,id,followers,videos_count,followers_per_video
0,6951881032375436293,19800,1,19800.0
1,6798230807473849350,15800,1,15800.0
2,6911061574435357698,24800,2,12400.0
3,81028050645106688,487800,67,7280.0
4,6948829470229300229,415000,63,6587.0
5,6714593971282641926,5804,1,5804.0
6,6808092341582267394,722300,153,4720.0
7,6762674634293019653,166100,36,4613.0
8,6964313100094866433,387900,90,4310.0
9,73008986968,63100,15,4206.0


### Find Most Efficient Influencers (Engagement per Video)

In [26]:
query = """
SELECT id, followers, videos_count, likes,
       ROUND((likes / NULLIF(videos_count, 0)), 2) AS likes_per_video,
       ROUND((awg_engagement_rate / NULLIF(videos_count, 0)), 4) AS engagement_per_video
FROM tiktok_profiles
ORDER BY engagement_per_video DESC
LIMIT 10;
"""

efficient_influencers = pd.read_sql(query, conn)
display(efficient_influencers)

,id,followers,videos_count,likes,likes_per_video,engagement_per_video
0,6761860606440375297,130,2,20800,10400.0,13.1483
1,6949875187790660613,1,1,7,7.0,3.4194
2,6951881032375436293,19800,1,134500,134500.0,3.2434
3,6989677079177397254,1,2,33,16.0,3.1893
4,7016420040141456389,3,2,71,35.0,1.9837
5,7061751047329989637,18,1,107,107.0,1.8302
6,7066131924145701894,1157,4,532300,133075.0,1.3758
7,7072780776455390214,12,3,471,157.0,1.3742
8,6926420836825121794,17,1,66,66.0,1.0985
9,6586992401892671494,277,3,8372,2790.0,1.0528


##### Helps brands pick influencers who drive the most engagement without spamming videos.


### Engagement Rate vs. Follower Growth (Identifying Rising Stars)

In [27]:
query = """
SELECT id, followers, videos_count, awg_engagement_rate,
       ROUND((followers / NULLIF(videos_count, 0)), 2) AS followers_per_video
FROM tiktok_profiles
ORDER BY followers_per_video DESC, awg_engagement_rate DESC
LIMIT 10;
"""
rising_stars = pd.read_sql(query, conn)
display(rising_stars)

,id,followers,videos_count,awg_engagement_rate,followers_per_video
0,6951881032375436293,19800,1,3.243409,19800.0
1,6798230807473849350,15800,1,1.019075,15800.0
2,6911061574435357698,24800,2,-0.185170,12400.0
3,81028050645106688,487800,67,-0.154826,7280.0
4,6948829470229300229,415000,63,-0.169266,6587.0
5,6714593971282641926,5804,1,-0.196628,5804.0
6,6808092341582267394,722300,153,-0.154103,4720.0
7,6762674634293019653,166100,36,-0.087358,4613.0
8,6964313100094866433,387900,90,-0.197101,4310.0
9,73008986968,63100,15,-0.191794,4206.0


###  Find the Best Regions for Influencer Marketing

In [28]:
query = """
SELECT region, 
       COUNT(id) AS num_influencers, 
       ROUND(AVG(awg_engagement_rate), 4) AS avg_engagement
FROM tiktok_profiles
GROUP BY region
ORDER BY avg_engagement DESC
LIMIT 5;
"""

best_regions = pd.read_sql(query, conn)
display(best_regions)

,region,num_influencers,avg_engagement
0,NO,2,1.6148
1,RO,4,0.9455
2,CL,3,0.8890
3,NP,2,0.8393
4,XK,1,0.7780


### Engagement Trends Across Language Groups

In [29]:
query = """
SELECT predicted_lang, 
       COUNT(id) AS num_influencers,
       ROUND(AVG(awg_engagement_rate), 4) AS avg_engagement
FROM tiktok_profiles
GROUP BY predicted_lang
ORDER BY avg_engagement DESC
LIMIT 5;
"""

lang_engagemen = pd.read_sql(query, conn)
display(best_regions)

,region,num_influencers,avg_engagement
0,NO,2,1.6148
1,RO,4,0.9455
2,CL,3,0.8890
3,NP,2,0.8393
4,XK,1,0.7780


### A/B Test: Do Verified Influencers Actually Perform Better?

In [30]:
query = """
WITH engagement_stats AS (
    SELECT is_verified, 
           COUNT(id) AS num_influencers,
           AVG(awg_engagement_rate) AS avg_engagement,
           STDDEV(awg_engagement_rate) AS std_dev
    FROM tiktok_profiles
    GROUP BY is_verified
)
SELECT is_verified, num_influencers, avg_engagement, std_dev,
       ROUND(avg_engagement / NULLIF(std_dev, 0), 4) AS z_score
FROM engagement_stats;
"""
verified_influencer = pd.read_sql(query, conn)
display(verified_influencer)

DatabaseError: Execution failed on sql '
WITH engagement_stats AS (
    SELECT is_verified, 
           COUNT(id) AS num_influencers,
           AVG(awg_engagement_rate) AS avg_engagement,
           STDDEV(awg_engagement_rate) AS std_dev
    FROM tiktok_profiles
    GROUP BY is_verified
)
SELECT is_verified, num_influencers, avg_engagement, std_dev,
       ROUND(avg_engagement / NULLIF(std_dev, 0), 4) AS z_score
FROM engagement_stats;
': no such function: STDDEV